In [1]:
from minobr.people import PeopleOccupation
from minobr.university.faculty import Faculty, FacultyTypes, FacultyManagement
from minobr.university.group import Group, Student
from minobr.university.staff import StaffType, Teacher
from minobr.university.semestr import Semester, SemesterException
from minobr.university.topics import Topic
from minobr.university.university import University

# Lets create test university

### Create university. Two faculties. Three groups 

In [2]:
u1 = University()
f1 = Faculty('Test Tech Faculty', FacultyTypes.tech)
f2 = Faculty('Test Tech Faculty 2', FacultyTypes.hum)
g1 = Group('Test group1')
g2 = Group('Test group2')
g3 = Group('Test group3')

### Append all objects

In [3]:
f1.add_group(g1)
f2.add_group(g2)
f2.add_group(g3)
u1.add(f1)
u1.add(f2)

### Add any students to groups

In [4]:
for i in range(1000):
    if i < 500:
        g = g1
    elif 500 < i < 800:
        g = g2
    else:
        g = g3
    p = Student(f1, g, name='test_'+unicode(i))
    p.add_employment(PeopleOccupation.student, '1st grade')
    g.add(p)


### Lets see what we have

In [5]:
u1.list()
print '\n'.join([unicode(f)+' '+unicode(g) for f,g in [(f,f.groups) for f in u1.list()]])

<Faculty humanitarian> Test Tech Faculty 2 [<Group Test group2> population: 299, <Group Test group3> population: 201]
<Faculty technical> Test Tech Faculty [<Group Test group1> population: 500]


### Add staff to faculty

In [6]:
f1.staff()[StaffType.teachers].list()

[]

In [7]:
t1 = Teacher(name='Sam', last_name='Osbourne', knowledge_area='math')
dean = Teacher(name='Dean', last_name='Osbourne', knowledge_area='manager')
f1.hire(StaffType.management, dean, 'dean') #  hack, head manager should set manualy
print t1, dean

<Teacher> Sam <Teacher> Dean


In [8]:
f_mngr = FacultyManagement(f1)
f_mngr.set_manager(dean)
f_mngr.hire(StaffType.teachers, t1, position='math_teacher')

In [9]:
f1.staff()[StaffType.teachers].list()

[<Teacher> Sam]

In [10]:
semester = Semester(course=1, number=1, faculty=f1); print semester

In [11]:
f1.add_semester(semester); f1.semesters

In [12]:
TOPICS = [
    {'math': ['algebra', 'mathan', 'geometry', 'discret_math', 'high_math']},
    {'lang': ['Russian', 'English']},
]
s_topics = [Topic(t_name, t1) for t_name in TOPICS[0]['math']]
for topic in s_topics:
    semester.add_topic(topic)

### Create students and enroll to semester

In [13]:
student1 = Student(faculty=f1, group=g1, name='Lian')
student2 = Student(faculty=f1, group=g1, name='Mark')
g1.add(student1)
semester.enroll(student1, s_topics[:len(s_topics)/2])
g1.add(student2)
semester.enroll(student2, s_topics[:len(s_topics)/2])
print semester.enrollments
print
print "Try to enroll less than 80% of semesters topics"
try:
    semester.enroll(student2, ['algebra'])
except SemesterException as e:
    print 'Exception: ', e

set([<minobr.university.semestr.Enrollment object at 0x7f625c5dd610>, <minobr.university.semestr.Enrollment object at 0x7f625c5e3190>])

Try to enroll less than 80% of semesters topics
Exception:  You must select topics from semester


### Change topic teacher


In [14]:
teacher2 = Teacher(name='Alex', last_name='Osbourne', knowledge_area='math')
teacher3 = Teacher(name='Kate', last_name='Osbourne', knowledge_area='lang')
topic = s_topics[0]
print topic
f_mngr.change_topic_teacher(topic, teacher2)
print topic
f_mngr.change_topic_teacher(topic, teacher3)
print topic, ';', teacher3, "is not applied, different knowledge area" 

<Topic> algebra by <Teacher> Sam
<Topic> algebra by <Teacher> Alex
<Topic> algebra by <Teacher> Alex ; <Teacher> Kate is not applied, different knowledge area


### new topic enrollment date is set by teacher

In [15]:
import datetime
topic_enroll_date = datetime.datetime.now()
teacher2.schedule_topic(g1, topic_enroll_date, topic)
print g1.get_timetable()


defaultdict(<type 'list'>, {u'<Topic> algebra by <Teacher> Alex': [<minobr.university.group.Schedule object at 0x7f62613e7190>]})


### After lesson finished, teacher checks who is enrolled

In [16]:
not_attended_student = student2
schedule_obj = teacher2.set_attending(g1, topic_enroll_date, topic, [student1], [not_attended_student])
print schedule_obj.is_attended(student1) == topic_enroll_date, schedule_obj.is_attended(student2) == topic_enroll_date
schedule_obj = teacher2.set_attending(g1, datetime.datetime.now(), topic, [student1], [not_attended_student])

True False


In [17]:
print f_mngr.students_to_fire(), not_attended_student in f_mngr.students_to_fire() 

[<Student> Mark] True


In [18]:
[sch.student_skipped for sch in g1.get_schedules(topic_enroll_date, topic)]

[[<Student> Mark], [<Student> Mark]]